In [112]:
from transformers import AutoModelForSequenceClassification, AutoTokenizer, AutoConfig
import torch
import numpy as np
from scipy.special import softmax
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
import pandas as pd

In [113]:
# Load your dataset
# Assuming your data is in the "twitter_sentiments" folder
train_path = "twitter_sentiments/twitter_training.csv"
validation_path = "twitter_sentiments/twitter_validation.csv"

train_df = pd.read_csv(train_path, encoding='utf-8')
validation_df = pd.read_csv(validation_path, encoding='utf-8')

In [114]:
train_df.head()

,2401,Borderlands,Positive,"im getting on borderlands and i will murder you all ,"
0,2401,Borderlands,Positive,I am coming to the borders and I will kill you...
1,2401,Borderlands,Positive,im getting on borderlands and i will kill you ...
2,2401,Borderlands,Positive,im coming on borderlands and i will murder you...
3,2401,Borderlands,Positive,im getting on borderlands 2 and i will murder ...
4,2401,Borderlands,Positive,im getting into borderlands and i can murder y...


In [115]:
# Preprocess text (username and link placeholders)
def preprocess(text):
    new_text = []
    for t in text.split(" "):
        t = '@user' if t.startswith('@') and len(t) > 1 else t
        t = 'http' if t.startswith('http') else t
        new_text.append(t)
    return " ".join(new_text)

In [116]:
# PT
MODEL = "cardiffnlp/twitter-roberta-base-sentiment-latest"
tokenizer = AutoTokenizer.from_pretrained(MODEL)
config = AutoConfig.from_pretrained(MODEL)
model = AutoModelForSequenceClassification.from_pretrained(MODEL, num_labels=3)

Some weights of the model checkpoint at cardiffnlp/twitter-roberta-base-sentiment-latest were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [117]:
# Training
# Assuming you have a column "sentiment" in your dataset
class_to_label = {"Negative": 0, "Neutral": 1, "Positive": 2}

# Drop rows with missing values in the "Tweet content" column
train_df = train_df.dropna(subset=[train_df.columns[2]])

# Map text labels to numerical labels
train_df["label"] = train_df.iloc[:, 2].map(class_to_label)

X_train, X_val, y_train, y_val = train_test_split(
    train_df.iloc[:, 2].apply(lambda x: preprocess(x) if pd.notna(x) else ""),  # Assuming the "Tweet content" column is at index 2
    train_df["label"],  # Use the new "label" column
    test_size=0.2,
    random_state=42
)

# Convert the labels to a list of numerical values
y_train = [class_to_label.get(label, -1) for label in y_train]

# Filter out instances where the label is not in class_to_label
X_train = [X_train[i] for i in range(len(y_train)) if y_train[i] != -1]
y_train = [label for label in y_train if label != -1]

In [118]:
optimizer = torch.optim.AdamW(model.parameters(), lr=5e-5)
loss_fn = torch.nn.CrossEntropyLoss()

num_epochs = 3

In [119]:
for epoch in range(num_epochs):
    model.train()
    for text, label in zip(X_train, y_train):
        encoded_input = tokenizer(text, return_tensors='pt')
        output = model(**encoded_input, labels=torch.tensor([label], dtype=torch.long))
        loss = output.loss
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

In [120]:
# Save the fine-tuned model
model.save_pretrained("fine_tuned_model")

# Testing
# Load the fine-tuned model
model = AutoModelForSequenceClassification.from_pretrained("fine_tuned_model")

df = df.dropna(subset=[df.columns[0]])
# Initialize lists to store true labels and predicted labels
true_labels = []
predicted_labels = []

# Set the model to evaluation mode
model.eval()

RobertaForSequenceClassification(
  (roberta): RobertaModel(
    (embeddings): RobertaEmbeddings(
      (word_embeddings): Embedding(50265, 768, padding_idx=1)
      (position_embeddings): Embedding(514, 768, padding_idx=1)
      (token_type_embeddings): Embedding(1, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): RobertaEncoder(
      (layer): ModuleList(
        (0-11): 12 x RobertaLayer(
          (attention): RobertaAttention(
            (self): RobertaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): RobertaSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
             

In [121]:
# Iterate over the validation dataset
for text, label in zip(X_val, y_val):
    text = preprocess(text)
    encoded_input = tokenizer(text, return_tensors='pt')
    output = model(**encoded_input)
    scores = output.logits.detach().numpy()
    scores = softmax(scores, axis=1)
    
    # Get the predicted label
    predicted_label_id = np.argmax(scores)
    predicted_label = config.id2label[predicted_label_id]
    
    # Append true and predicted labels to lists
    true_labels.append(label)
    predicted_labels.append(predicted_label)

In [131]:
# Convert true_labels and predicted_labels to numpy arrays
true_labels_np = np.array(true_labels)
predicted_labels_np = np.array(predicted_labels)

# print(type(true_labels_np))
# print(true_labels_np)
# print(predicted_labels_np)
# print(len(true_labels_np))
# print(len(predicted_labels_np))


nan_indices = np.isnan(true_labels_np)
print(nan_indices)

# Remove NaN values from both true_labels_np and predicted_labels_np
true_labels_np = true_labels_np[~nan_indices]
predicted_labels_np = predicted_labels_np[~nan_indices]


# print("After removing")
# print(len(true_labels_np))
# print(len(predicted_labels_np))


# Convert string labels to numeric values
label_mapping = {"positive": 2, "neutral": 1, "negative": 0}

# Convert string labels to numeric values for predicted_labels_np
predicted_labels_np = np.array([label_mapping.get(label, label) for label in predicted_labels_np])
print(predicted_labels_np)

# Calculate accuracy
accuracy = accuracy_score(true_labels_np, predicted_labels_np)
print(f"Accuracy: {accuracy * 100:.2f}%")

[nan  2.  0. ...  2.  0.  1.]
['negative' 'positive' 'neutral' ... 'positive' 'neutral' 'neutral']
14937
14937
[ True False False ... False False False]
After removing
12276
12276
[2 1 2 ... 2 1 1]
Accuracy: 63.58%
